In [70]:
import pandas as pd
import numpy as np
import plotly.express as px

from openpyxl import Workbook, load_workbook
from xlsxwriter import Workbook

import requests
from datetime import datetime
from pytz import timezone
import pytz

import warnings
warnings.filterwarnings('ignore')

def dia():
    from datetime import date, timedelta
    dia = date.today() + timedelta(0)
    return dia

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df
    
def grafico(df, nome):
    df = df.reset_index(drop=True)
    df.index += 1
    df['Profit_acu'] = df.Profit.cumsum()
    profit = round(df.Profit_acu.tail(1).item(),2)
    ROI = round((df.Profit_acu.tail(1)/len(df)*100).item(),2)
    df.Profit_acu.plot(title=nome, xlabel='Entradas', ylabel='Stakes')
    print("Profit:",profit,"stakes em", len(df),"jogos")
    print("ROI:",ROI,"%")

In [71]:

df = pd.read_csv("https://github.com/rpthiago/ARKAD/blob/main/BASE_DE_DAD0S_FLASHSCORE/Base_FlahScore.csv?raw=true")

df = df.sort_values(['Date'])

In [72]:

df['HT_Total_Goals'] = df['HT_Goals_H'] + df['HT_Goals_A']

flt = ((df.FT_Odd_H != 0) & (df.FT_Odd_D != 0) & (df.FT_Odd_A != 0))
df = df[flt]

df["Total_Goals"] = df["FT_Goals_H"] + df["FT_Goals_A"]

df.loc[df['FT_Goals_H'] > df['FT_Goals_A'], 'FT_Result'] = "H"
df.loc[df['FT_Goals_H'] == df['FT_Goals_A'], 'FT_Result'] = "D"
df.loc[df['FT_Goals_H'] < df['FT_Goals_A'], 'FT_Result'] = "A"

# Profit
green = df.FT_Odd_D - 1
red = -1

df.loc[df['FT_Result'] == "D", 'Profit_back_draw'] = green
df.loc[df['FT_Result'] != "D", 'Profit_back_draw'] = red

green = df.FT_Odd_H - 1
red = -1

df.loc[df['FT_Result'] == "H", 'Profit_back_home'] = green
df.loc[df['FT_Result'] != "H", 'Profit_back_home'] = red


green = df.FT_Odd_A - 1
red = -1

df.loc[df['FT_Result'] == "A", 'Profit_back_away'] = green
df.loc[df['FT_Result'] != "A", 'Profit_back_away'] = red


# Probabilidades
df['p_H'] = 1 / df.FT_Odd_H
df['p_D'] = 1 / df.FT_Odd_D
df['p_A'] = 1 / df.FT_Odd_A

# CV das Odds do Match Odds
df['CV_HDA'] = df[['p_H','p_D','p_A']].std(ddof=0, axis=1) / df[['p_H','p_D','p_A']].mean(axis=1)

# Pontos
df.loc[df['FT_Goals_H'] > df['FT_Goals_A'], 'Ptos_H'] = 3
df.loc[df['FT_Goals_H'] == df['FT_Goals_A'], 'Ptos_H'] = 1
df.loc[df['FT_Goals_H'] < df['FT_Goals_A'], 'Ptos_H'] = 0

df.loc[df['FT_Goals_H'] < df['FT_Goals_A'], 'Ptos_A'] = 3
df.loc[df['FT_Goals_H'] == df['FT_Goals_A'], 'Ptos_A'] = 1
df.loc[df['FT_Goals_H'] > df['FT_Goals_A'], 'Ptos_A'] = 0

df["Ptos_H"] = df["Ptos_H"].astype('int64')
df["Ptos_A"] = df["Ptos_A"].astype('int64')

df['Media_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
df['Media_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

df['Media_Ptos_H'] = df.groupby('Home')['Media_Ptos_H'].transform('last')
df['Media_Ptos_A'] = df.groupby('Away')['Media_Ptos_A'].transform('last')

df['DesvPad_Ptos_H'] = df.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
df['DesvPad_Ptos_A'] = df.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

df['DesvPad_Ptos_H'] = df.groupby('Home')['DesvPad_Ptos_H'].transform('last')
df['DesvPad_Ptos_A'] = df.groupby('Away')['DesvPad_Ptos_A'].transform('last')

df['CV_Ptos_H'] = df['DesvPad_Ptos_H'] / df['Media_Ptos_H']
df['CV_Ptos_A'] = df['DesvPad_Ptos_A'] / df['Media_Ptos_A']

# Valor do Gol
df['VG_H'] = df.FT_Goals_H * df.p_A
df['VG_A'] = df.FT_Goals_A * df.p_H

df['Media_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
df['Media_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

df['Media_VG_H'] = df.groupby('Home')['Media_VG_H'].transform('last')
df['Media_VG_A'] = df.groupby('Away')['Media_VG_A'].transform('last')

df['DesvPad_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
df['DesvPad_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

df['DesvPad_VG_H'] = df.groupby('Home')['DesvPad_VG_H'].transform('last')
df['DesvPad_VG_A'] = df.groupby('Away')['DesvPad_VG_A'].transform('last')

df['CV_VG_H'] = df['DesvPad_VG_H'] / df['Media_VG_H']
df['CV_VG_A'] = df['DesvPad_VG_A'] / df['Media_VG_A']

# Custo do Gol
df['CG_H'] = df.p_H / df.FT_Goals_H
df['CG_A'] = df.p_A / df.FT_Goals_A

df['Media_CG_H'] = df.groupby('Home')['CG_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
df['Media_CG_A'] = df.groupby('Away')['CG_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

df['Media_CG_H'] = df.groupby('Home')['Media_CG_H'].transform('last')
df['Media_CG_A'] = df.groupby('Away')['Media_CG_A'].transform('last')

df['DesvPad_CG_H'] = df.groupby('Home')['CG_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
df['DesvPad_CG_A'] = df.groupby('Away')['CG_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

df['DesvPad_CG_H'] = df.groupby('Home')['DesvPad_CG_H'].transform('last')
df['DesvPad_CG_A'] = df.groupby('Away')['DesvPad_CG_A'].transform('last')

df['CV_CG_H'] = df['DesvPad_CG_H'] / df['Media_CG_H']
df['CV_CG_A'] = df['DesvPad_CG_A'] / df['Media_CG_A']


df.replace(np.inf, 1, inplace=True)
df = drop_reset_index(df)
print('pronto')

pronto


In [73]:
# media de gols 5 jogos

df['mean55_gols_feitos_home_ft'] = df.groupby('Home')['FT_Goals_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)
df['mean55_gols_sofridos_home_ft'] = df.groupby('Home')['FT_Goals_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)

df['mean55_gols_feitos_home_ft'] = df.groupby('Home')['mean55_gols_feitos_home_ft'].transform('last')
df['mean55_gols_sofridos_home_ft'] = df.groupby('Home')['mean55_gols_sofridos_home_ft'].transform('last')

df['mean55_gols_feitos_away_ft'] = df.groupby('Away')['FT_Goals_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)
df['mean55_gols_sofridos_away_ft'] = df.groupby('Away')['FT_Goals_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)

df['mean55_gols_feitos_away_ft'] = df.groupby('Away')['mean55_gols_feitos_away_ft'].transform('last')
df['mean55_gols_sofridos_away_ft'] = df.groupby('Away')['mean55_gols_sofridos_away_ft'].transform('last')

# desvio padrao 5 jogos

df['std5_gols_feitos_home_ft'] = df.groupby('Home')['FT_Goals_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)
df['std5_gols_sofridos_home_ft'] = df.groupby('Home')['FT_Goals_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)

df['std5_gols_feitos_home_ft'] = df.groupby('Home')['std5_gols_feitos_home_ft'].transform('last')
df['std5_gols_sofridos_home_ft'] = df.groupby('Home')['std5_gols_sofridos_home_ft'].transform('last')

df['std5_gols_feitos_away_ft'] = df.groupby('Away')['FT_Goals_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)
df['std5_gols_sofridos_away_ft'] = df.groupby('Away')['FT_Goals_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)

df['std5_gols_feitos_away_ft'] = df.groupby('Away')['std5_gols_feitos_away_ft'].transform('last')
df['std5_gols_sofridos_away_ft'] = df.groupby('Away')['std5_gols_sofridos_away_ft'].transform('last')

# coeficiente de variação

df['cv5_feitos_home_ft'] = df['std5_gols_feitos_home_ft'] / df['mean55_gols_feitos_home_ft']
df['cv5_sofridos_home_ft'] = df['std5_gols_sofridos_home_ft'] / df['mean55_gols_sofridos_home_ft']

df['cv5_feitos_home_ft'] = df.groupby('Home')['cv5_feitos_home_ft'].transform('last')
df['cv5_sofridos_home_ft'] = df.groupby('Home')['cv5_sofridos_home_ft'].transform('last')

df['cv5_feitos_away_ft'] = df['std5_gols_feitos_away_ft'] / df['mean55_gols_feitos_away_ft']
df['cv5_sofridos_away_ft'] = df['std5_gols_sofridos_away_ft'] / df['mean55_gols_sofridos_away_ft']

df['cv5_feitos_away_ft'] = df.groupby('Away')['cv5_feitos_away_ft'].transform('last')
df['cv5_sofridos_away_ft'] = df.groupby('Away')['cv5_sofridos_away_ft'].transform('last')
df.dropna(inplace=True)

print('pronto')

pronto


In [74]:
# media de gols 10 jogos

df['mean10_gols_feitos_home_ft'] = df.groupby('Home')['FT_Goals_H'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True)
df['mean10_gols_sofridos_home_ft'] = df.groupby('Home')['FT_Goals_A'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True)

df['mean10_gols_feitos_home_ft'] = df.groupby('Home')['mean10_gols_feitos_home_ft'].transform('last')
df['mean10_gols_sofridos_home_ft'] = df.groupby('Home')['mean10_gols_sofridos_home_ft'].transform('last')

df['mean10_gols_feitos_away_ft'] = df.groupby('Away')['FT_Goals_A'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True)
df['mean10_gols_sofridos_away_ft'] = df.groupby('Away')['FT_Goals_H'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True)

df['mean10_gols_feitos_away_ft'] = df.groupby('Away')['mean10_gols_feitos_away_ft'].transform('last')
df['mean10_gols_sofridos_away_ft'] = df.groupby('Away')['mean10_gols_sofridos_away_ft'].transform('last')

# desvio padrao 10 jogos

df['std10_gols_feitos_home_ft'] = df.groupby('Home')['FT_Goals_H'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True)
df['std10_gols_sofridos_home_ft'] = df.groupby('Home')['FT_Goals_A'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True)

df['std10_gols_feitos_home_ft'] = df.groupby('Home')['std10_gols_feitos_home_ft'].transform('last')
df['std10_gols_sofridos_home_ft'] = df.groupby('Home')['std10_gols_sofridos_home_ft'].transform('last')

df['std10_gols_feitos_away_ft'] = df.groupby('Away')['FT_Goals_A'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True)
df['std10_gols_sofridos_away_ft'] = df.groupby('Away')['FT_Goals_H'].rolling(window=10, min_periods=1).std().reset_index(0,drop=True)

df['std10_gols_feitos_away_ft'] = df.groupby('Away')['std10_gols_feitos_away_ft'].transform('last')
df['std10_gols_sofridos_away_ft'] = df.groupby('Away')['std10_gols_sofridos_away_ft'].transform('last')

# coeficiente de variação 10 jogos

df['cv10_feitos_home_ft'] = df['std10_gols_feitos_home_ft'] / df['mean10_gols_feitos_home_ft']
df['cv10_sofridos_home_ft'] = df['std10_gols_sofridos_home_ft'] / df['mean10_gols_sofridos_home_ft']

df['cv10_feitos_home_ft'] = df.groupby('Home')['cv10_feitos_home_ft'].transform('last')
df['cv10_sofridos_home_ft'] = df.groupby('Home')['cv10_sofridos_home_ft'].transform('last')

df['cv10_feitos_away_ft'] = df['std10_gols_feitos_away_ft'] / df['mean10_gols_feitos_away_ft']
df['cv10_sofridos_away_ft'] = df['std10_gols_sofridos_away_ft'] / df['mean10_gols_sofridos_away_ft']

df['cv10_feitos_away_ft'] = df.groupby('Away')['cv10_feitos_away_ft'].transform('last')
df['cv10_sofridos_away_ft'] = df.groupby('Away')['cv10_sofridos_away_ft'].transform('last')

df.dropna(inplace=True)
print('pronto')


pronto


In [75]:
df['over05ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 1) else 0, axis=1)

df['over05ht'] = df.apply(lambda row: 1 if (row['HT_Total_Goals'] >= 1) else 0, axis=1)

df['over15ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 2) else 0, axis=1)

df['over25ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 3) else 0, axis=1)

df['over35ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 4) else 0, axis=1)

df['over45ft'] = df.apply(lambda row: 1 if (row['Total_Goals'] >= 5) else 0, axis=1) 

df['btts_ft'] = df.apply(lambda row: 1 if (row['FT_Goals_H'] > 0 and row['FT_Goals_A'] > 0 ) else 0, axis=1)

print('pronto')

pronto


In [76]:
# percentual mercado de gols 5 jogos

df['avg5_home_05ft'] = (df.groupby('Home')['over05ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_05ft'] = df.groupby('Home')['avg5_home_05ft'].transform('last')

df['avg5_away_05ft'] = (df.groupby('Away')['over05ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_05ft'] = df.groupby('Away')['avg5_away_05ft'].transform('last')

df['avg5_home_05ht'] = (df.groupby('Home')['over05ht'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_05ht'] = df.groupby('Home')['avg5_home_05ht'].transform('last')

df['avg5_away_05ht'] = (df.groupby('Away')['over05ht'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_05ht'] = df.groupby('Away')['avg5_away_05ht'].transform('last')

df['avg5_home_15ft'] = (df.groupby('Home')['over15ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_15ft'] = df.groupby('Home')['avg5_home_15ft'].transform('last')

df['avg5_away_15ft'] = (df.groupby('Away')['over15ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_15ft'] = df.groupby('Away')['avg5_away_15ft'].transform('last')

df['avg5_home_25ft'] = (df.groupby('Home')['over25ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_25ft'] = df.groupby('Home')['avg5_home_25ft'].transform('last')

df['avg5_away_25ft'] = (df.groupby('Away')['over25ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_25ft'] = df.groupby('Away')['avg5_away_25ft'].transform('last')

df['avg5_home_35ft'] = (df.groupby('Home')['over35ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_35ft'] = df.groupby('Home')['avg5_home_35ft'].transform('last')

df['avg5_away_35ft'] = (df.groupby('Away')['over35ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_35ft'] = df.groupby('Away')['avg5_away_35ft'].transform('last')

df['avg5_home_45ft'] = (df.groupby('Home')['over45ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_45ft'] = df.groupby('Home')['avg5_home_45ft'].transform('last')

df['avg5_away_45ft'] = (df.groupby('Away')['over45ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_45ft'] = df.groupby('Away')['avg5_away_45ft'].transform('last')

df['avg5_home_btts_ft'] = (df.groupby('Home')['btts_ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_home_btts_ft'] = df.groupby('Home')['avg5_home_btts_ft'].transform('last')

df['avg5_away_btts_ft'] = (df.groupby('Away')['btts_ft'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg5_away_btts_ft'] = df.groupby('Away')['avg5_away_btts_ft'].transform('last')

# percentual mercado de gols 10 jogos

df['avg10_home_05ft'] = (df.groupby('Home')['over05ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_05ft'] = df.groupby('Home')['avg10_home_05ft'].transform('last')

df['avg10_away_05ft'] = (df.groupby('Away')['over05ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_05ft'] = df.groupby('Away')['avg10_away_05ft'].transform('last')

df['avg10_home_05ht'] = (df.groupby('Home')['over05ht'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_05ht'] = df.groupby('Home')['avg10_home_05ht'].transform('last')

df['avg10_away_05ht'] = (df.groupby('Away')['over05ht'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_05ht'] = df.groupby('Away')['avg10_away_05ht'].transform('last')

df['avg10_home_15ft'] = (df.groupby('Home')['over15ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_15ft'] = df.groupby('Home')['avg10_home_15ft'].transform('last')

df['avg10_away_15ft'] = (df.groupby('Away')['over15ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_15ft'] = df.groupby('Away')['avg10_away_15ft'].transform('last')

df['avg10_home_25ft'] = (df.groupby('Home')['over25ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_25ft'] = df.groupby('Home')['avg10_home_25ft'].transform('last')

df['avg10_away_25ft'] = (df.groupby('Away')['over25ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_25ft'] = df.groupby('Away')['avg10_away_25ft'].transform('last')

df['avg10_home_35ft'] = (df.groupby('Home')['over35ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_35ft'] = df.groupby('Home')['avg10_home_35ft'].transform('last')

df['avg10_away_35ft'] = (df.groupby('Away')['over35ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_35ft'] = df.groupby('Away')['avg10_away_35ft'].transform('last')

df['avg10_home_45ft'] = (df.groupby('Home')['over45ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_45ft'] = df.groupby('Home')['avg10_home_45ft'].transform('last')

df['avg10_away_45ft'] = (df.groupby('Away')['over45ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_45ft'] = df.groupby('Away')['avg10_away_45ft'].transform('last')

df['avg10_home_btts_ft'] = (df.groupby('Home')['btts_ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_home_btts_ft'] = df.groupby('Home')['avg10_home_btts_ft'].transform('last')

df['avg10_away_btts_ft'] = (df.groupby('Away')['btts_ft'].rolling(window=10, min_periods=1).mean().reset_index(0,drop=True) * 100)
df['avg10_away_btts_ft'] = df.groupby('Away')['avg10_away_btts_ft'].transform('last')
df.dropna(inplace=True)

print('pronto')



pronto


In [77]:
df['coeficiente_pontos'] = df['Media_Ptos_H'] - df['Media_Ptos_A']
df['cv_pontos'] = df['DesvPad_Ptos_H'] + df['DesvPad_Ptos_A']

df['coeficiente10_gols_feitos'] = df['mean10_gols_feitos_home_ft'] - df['mean10_gols_feitos_away_ft']
df['cv10_gols_feitos'] = df['std10_gols_feitos_home_ft'] + df['std10_gols_feitos_away_ft']

df['coeficiente10_gols_sofridos'] = df['mean10_gols_sofridos_home_ft'] - df['mean10_gols_sofridos_away_ft']

df['cv_pontos_intervalos'] = pd.qcut(df.cv_pontos, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['cv_pontos_intervalos'], inplace=True)
df['cv_pontos_intervalos'] = df['cv_pontos_intervalos'].astype('int64')

df['cv_valordogol_home_intervalos'] = pd.qcut(df.CV_VG_H, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['cv_valordogol_home_intervalos'], inplace=True)
df['cv_valordogol_home_intervalos'] = df['cv_valordogol_home_intervalos'].astype(float)

df['cv_valordogol_away_intervalos'] = pd.qcut(df.CV_VG_A, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['cv_valordogol_away_intervalos'], inplace=True)
df['cv_valordogol_away_intervalos'] = df['cv_valordogol_away_intervalos'].astype(float)

df['valordogol_home_intervalos'] = pd.qcut(df.Media_VG_H, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['valordogol_home_intervalos'], inplace=True)
df['valordogol_home_intervalos'] = df['valordogol_home_intervalos'].astype('int64')

df['valordogol_away_intervalos'] = pd.qcut(df.Media_VG_A, q=6, labels=['1', '2', '3', '4', '5', '6'])
df.dropna(subset=['valordogol_away_intervalos'], inplace=True)
df['valordogol_away_intervalos'] = df['valordogol_away_intervalos'].astype('int64')

print('pronto')


pronto


In [78]:
df.replace(np.inf, 1, inplace=True)
df = drop_reset_index(df)

In [79]:
nova_ordem = ['Date', 'League', 'Home', 'Away','Season','FT_Odd_H','FT_Odd_D','FT_Odd_A','HT_Goals_H','HT_Goals_A',
              'HT_Total_Goals','FT_Goals_H','FT_Goals_A','Total_Goals','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_No',
              'FT_Odd_BTTS_Yes','p_H','p_D','p_A','CV_HDA','Media_Ptos_H','Media_Ptos_A','CV_Ptos_H','CV_Ptos_A','VG_H','VG_A',
              'Media_VG_H','Media_VG_A','CV_VG_H','CV_VG_A','CG_H','CG_A','Media_CG_H','Media_CG_A','CV_CG_H','CV_CG_A',
              'coeficiente10_gols_feitos',
              'cv10_gols_feitos','cv5_feitos_home_ft','cv5_sofridos_home_ft','coeficiente_pontos',
              'cv_pontos', 'cv_pontos_intervalos', 'mean55_gols_feitos_home_ft','mean55_gols_sofridos_home_ft',
              'mean55_gols_feitos_away_ft',
              'mean55_gols_sofridos_away_ft','mean10_gols_feitos_home_ft','mean10_gols_sofridos_home_ft',
              'mean10_gols_feitos_away_ft',
              'mean10_gols_sofridos_away_ft', 'avg5_home_05ft','avg5_home_05ht','avg5_home_15ft',
              'avg5_home_25ft', 'avg5_home_35ft', 'avg5_home_45ft','avg10_home_05ft','avg10_home_05ht',
              'avg10_home_15ft','avg10_home_25ft','avg10_away_25ft','avg10_away_35ft','avg10_home_35ft','avg10_home_45ft',
              'avg10_away_45ft','avg5_away_05ft','avg5_away_05ht','avg5_away_15ft', 'avg5_away_25ft', 
              'avg5_away_35ft', 'avg5_away_45ft', 'avg5_away_btts_ft', 'avg10_away_05ft','avg10_away_05ht',
              'avg10_away_15ft','avg10_away_btts_ft', 'avg10_home_btts_ft', 
              'avg5_home_btts_ft', 'cv5_feitos_away_ft', 'cv5_sofridos_away_ft', 'coeficiente10_gols_sofridos',
              'cv_valordogol_away_intervalos', 'cv_valordogol_home_intervalos', 'valordogol_home_intervalos',
              'valordogol_away_intervalos', 'cv10_feitos_home_ft', 'cv10_sofridos_away_ft', 'cv10_feitos_away_ft',
              'cv10_sofridos_home_ft']

df = df[nova_ordem]




In [80]:
#df.to_csv(f'jogos_estatistica.csv', index = False)

In [81]:
base_H = df[['Home', 'Media_Ptos_H', 'CV_Ptos_H', 'Media_VG_H','CV_VG_H', 'Media_CG_H', 'CV_CG_H', 'cv5_feitos_home_ft', 'cv5_sofridos_home_ft',
              'coeficiente10_gols_feitos', 'cv10_gols_feitos',  'coeficiente_pontos',  'cv_pontos', 'cv_pontos_intervalos',            
              'mean55_gols_feitos_home_ft', 'mean55_gols_sofridos_home_ft', 'mean10_gols_feitos_home_ft', 'mean10_gols_sofridos_home_ft',
              'avg5_home_05ft',  'avg5_home_05ht',  'avg5_home_15ft',  'avg5_home_25ft', 'avg5_home_35ft', 'avg5_home_45ft', 'avg10_home_05ft',
              'avg10_home_05ht',  'avg10_home_15ft', 'avg5_home_btts_ft',
              'avg10_home_25ft', 'avg10_away_25ft', 'avg10_away_35ft', 'avg10_home_35ft', 'avg10_home_45ft', 'avg10_home_btts_ft',
              'cv_valordogol_home_intervalos', 'valordogol_home_intervalos', 'cv10_feitos_home_ft', 'cv10_sofridos_home_ft']]

base_A = df[['Away', 'Media_Ptos_A','CV_Ptos_A', 'Media_VG_A','CV_VG_A', 'Media_CG_A', 'CV_CG_A', 'avg10_away_25ft', 'avg10_away_35ft',
               'avg10_away_45ft',
               'coeficiente10_gols_sofridos', 'mean55_gols_feitos_away_ft', 'mean55_gols_sofridos_away_ft', 'mean10_gols_feitos_away_ft',
               'mean10_gols_sofridos_away_ft',  'avg5_away_05ft',  'avg5_away_05ht','avg5_away_25ft', 'avg5_away_15ft',
               'avg5_away_35ft', 'avg5_away_45ft', 'avg10_away_05ft', 'cv5_feitos_away_ft', 
               'avg10_away_05ht',  'avg10_away_15ft',  'avg10_away_btts_ft', 'avg5_away_btts_ft', 'cv_valordogol_away_intervalos',
               'valordogol_away_intervalos', 'cv10_sofridos_away_ft', 'cv10_feitos_away_ft', 'cv5_sofridos_away_ft']]

In [82]:
from datetime import date, datetime, timedelta
data = date.today() + timedelta(0)
print(data)

#jogos_do_dia = pd.read_csv('https://github.com/futpythontrader/YouTube/blob/main/Jogos_do_Dia_FlashScore/'+str(dia)+'_Jogos_do_Dia_FlashScore.csv?raw=true')
jogos_do_dia = pd.read_csv('C:/Users/thiag/OneDrive/curso python/modelo_leandro_2/flashscore/Jogos_do_Dia/'+str(data)+'_Jogos_do_Dia.csv')



jogos_do_dia = jogos_do_dia[['League','Round','Date','Time','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']]
#jogos_do_dia.columns = ['League','Round','Date','Time','Home','Away','FT_Odds_H','FT_Odds_D','FT_Odds_A','FT_Odds_Over25','FT_Odds_Under25','Odds_BTTS_Yes','Odds_BTTS_No']
jogos_do_dia.dropna(inplace=True)
jogos_do_dia.reset_index(inplace=True, drop=True)
jogos_do_dia.index = jogos_do_dia.index.set_names(['Nº'])
Jogos_do_Dia = jogos_do_dia.rename(index=lambda x: x + 1)
##Jogos_do_Dia

2023-07-24


In [83]:
Jogos_do_Dia

,League,Round,Date,Time,Home,Away,FT_Odd_H,FT_Odd_D,FT_Odd_A,FT_Odd_Over25,FT_Odd_Under25,FT_Odd_BTTS_Yes,FT_Odd_BTTS_No
Nº,,,,,,,,,,,,,
1,ARGENTINA - LIGA PROFESIONAL,ROUND 26,24/07/2023,16:30,Talleres Cordoba,Gimnasia L.P.,1.53,4.00,7.00,2.05,1.80,2.10,1.67
2,ARGENTINA - LIGA PROFESIONAL,ROUND 26,24/07/2023,16:30,Velez Sarsfield,Union de Santa Fe,2.25,2.90,3.80,2.88,1.40,2.25,1.57
3,ARGENTINA - LIGA PROFESIONAL,ROUND 26,24/07/2023,18:30,Banfield,Godoy Cruz,2.30,3.10,3.40,2.50,1.50,2.10,1.67
4,ARGENTINA - LIGA PROFESIONAL,ROUND 26,24/07/2023,19:30,Racing Club,Central Cordoba,1.91,3.40,4.33,2.20,1.65,2.00,1.75
5,ARGENTINA - LIGA PROFESIONAL,ROUND 26,24/07/2023,20:45,Boca Juniors,Newells Old Boys,1.95,3.20,4.50,2.88,1.40,2.38,1.53
6,ARGENTINA - PRIMERA NACIONAL,ROUND 23,24/07/2023,18:00,Aldosivi,Brown Adrogue,2.50,3.10,3.10,2.50,1.50,2.10,1.67
7,ARGENTINA - PRIMERA NACIONAL,ROUND 26,24/07/2023,21:15,San Martin S.J.,Def. de Belgrano,2.10,3.10,4.00,2.88,1.40,2.50,1.50
8,BOLIVIA - DIVISION PROFESIONAL,ROUND 20,24/07/2023,16:00,Always Ready,Vaca Diez,1.17,8.00,14.00,1.40,2.88,1.91,1.80
9,BOLIVIA - DIVISION PROFESIONAL,ROUND 20,24/07/2023,19:00,Universitario de Vinto,Blooming,1.91,3.60,4.00,1.98,1.83,1.80,1.91


In [84]:
lista=[]

for a,b,c,d,e,f,g,h,i,j,k,l in zip(Jogos_do_Dia.League,Jogos_do_Dia.Date,Jogos_do_Dia.Time,Jogos_do_Dia.Home,Jogos_do_Dia.Away,
                                   Jogos_do_Dia.FT_Odd_H,Jogos_do_Dia.FT_Odd_D,Jogos_do_Dia.FT_Odd_A,Jogos_do_Dia.FT_Odd_Over25,Jogos_do_Dia.FT_Odd_Under25,
                                   Jogos_do_Dia.FT_Odd_BTTS_Yes,Jogos_do_Dia.FT_Odd_BTTS_No):
        League = a
        Date = b
        Time = c 
        home = d
        away = e
        FT_Odd_H = f
        FT_Odd_D = g
        FT_Odd_A = h
        FT_Odd_Over25 = i
        FT_Odd_Under25 = j
        FT_Odd_BTTS_Yes = k
        FT_Odd_BTTS_No = l
  
        df1 = base_H[base_H.Home == home].tail(1)

        df2 = base_A[base_A.Away == away].tail(1)

        jogo = {}

        jogo["League"] = League 
        jogo["Date"] = Date
        jogo["Time"] = Time
        jogo["Home"] = home
        jogo["Away"] = away
        jogo["FT_Odd_H"] = FT_Odd_H
        jogo["FT_Odd_D"] = FT_Odd_D
        jogo["FT_Odd_A"] = FT_Odd_A
        jogo["FT_Odd_Over25"] = FT_Odd_Over25
        jogo["FT_Odd_Under25"] = FT_Odd_Under25
        jogo["FT_Odd_BTTS_Yes"] = FT_Odd_BTTS_Yes
        jogo["FT_Odd_BTTS_No"] = FT_Odd_BTTS_No

        
        try:
        
            jogo['Media_Ptos_H'] = df1[df1.Home == home]['Media_Ptos_H'].iloc[0]
            jogo['CV_Ptos_H'] = df1[df1.Home == home]['CV_Ptos_H'].iloc[0]
            jogo['Media_VG_H'] = df1[df1.Home == home]['Media_VG_H'].iloc[0]
            jogo['CV_VG_H'] = df1[df1.Home == home]['CV_VG_H'].iloc[0]
            jogo['Media_CG_H'] = df1[df1.Home == home]['Media_CG_H'].iloc[0]
            jogo['CV_CG_H'] = df1[df1.Home == home]['CV_CG_H'].iloc[0]
            jogo['mean55_gols_feitos_home_ft'] = df1[df1.Home == home]['mean55_gols_feitos_home_ft'].iloc[0]
            jogo['cv5_feitos_home_ft'] = df1[df1.Home == home]['cv5_feitos_home_ft'].iloc[0]
            jogo['mean55_gols_sofridos_home_ft'] = df1[df1.Home == home]['mean55_gols_sofridos_home_ft'].iloc[0]
            jogo['cv5_sofridos_home_ft'] = df1[df1.Home == home]['cv5_sofridos_home_ft'].iloc[0]
            jogo['avg10_home_05ft'] = df1[df1.Home == home]['avg10_home_05ft'].iloc[0]
            jogo['avg10_home_05ht'] = df1[df1.Home == home]['avg10_home_05ht'].iloc[0]
            jogo['avg10_home_15ft'] = df1[df1.Home == home]['avg10_home_15ft'].iloc[0]
            jogo['avg10_home_25ft'] = df1[df1.Home == home]['avg10_home_25ft'].iloc[0]
            jogo['avg10_home_35ft'] = df1[df1.Home == home]['avg10_home_35ft'].iloc[0]
            jogo['avg10_home_45ft'] = df1[df1.Home == home]['avg10_home_45ft'].iloc[0]
            jogo['avg10_home_btts_ft'] = df1[df1.Home == home]['avg10_home_btts_ft'].iloc[0]
            jogo['mean10_gols_feitos_home_ft'] = df1[df1.Home == home]['mean10_gols_feitos_home_ft'].iloc[0]
            jogo['cv10_feitos_home_ft'] = df1[df1.Home == home]['cv10_feitos_home_ft'].iloc[0]
            jogo['mean10_gols_sofridos_home_ft'] = df1[df1.Home == home]['mean10_gols_sofridos_home_ft'].iloc[0]
            jogo['cv10_sofridos_home_ft'] = df1[df1.Home == home]['cv10_sofridos_home_ft'].iloc[0]
            jogo['avg5_home_05ft'] = df1[df1.Home == home]['avg5_home_05ft'].iloc[0]
            jogo['avg10_home_btts_ft'] = df1[df1.Home == home]['avg10_home_btts_ft'].iloc[0]
            jogo['avg5_home_05ht'] = df1[df1.Home == home]['avg5_home_05ht'].iloc[0]
            jogo['avg5_home_15ft'] = df1[df1.Home == home]['avg5_home_15ft'].iloc[0]
            jogo['avg5_home_25ft'] = df1[df1.Home == home]['avg5_home_25ft'].iloc[0]
            jogo['avg5_home_35ft'] = df1[df1.Home == home]['avg5_home_35ft'].iloc[0]
            jogo['avg5_home_45ft'] = df1[df1.Home == home]['avg5_home_45ft'].iloc[0]
            jogo['avg5_home_btts_ft'] = df1[df1.Home == home]['avg5_home_btts_ft'].iloc[0]
            jogo['cv_valordogol_home_intervalos'] = df1[df1.Home == home]['cv_valordogol_home_intervalos'].iloc[0]
            jogo['valordogol_home_intervalos'] = df1[df1.Home == home]['valordogol_home_intervalos'].iloc[0]
            jogo['cv_pontos_intervalos'] = df1[df1.Home == home]['cv_pontos_intervalos'].iloc[0]
            jogo['cv_pontos'] = df1[df1.Home == home]['cv_pontos'].iloc[0]
            jogo['coeficiente_pontos'] = df1[df1.Home == home]['coeficiente_pontos'].iloc[0]
            jogo['cv10_gols_feitos'] = df1[df1.Home == home]['cv10_gols_feitos'].iloc[0]
            jogo['coeficiente10_gols_feitos'] = df1[df1.Home == home]['coeficiente10_gols_feitos'].iloc[0]
           
        
        
            jogo['mean55_gols_feitos_away_ft'] = df2[df2.Away == away]['mean55_gols_feitos_away_ft'].iloc[0]
            jogo['cv5_feitos_away_ft'] = df2[df2.Away == away]['cv5_feitos_away_ft'].iloc[0]
            jogo['mean55_gols_sofridos_away_ft'] = df2[df2.Away == away]['mean55_gols_sofridos_away_ft'].iloc[0]
            jogo['cv5_sofridos_away_ft'] = df2[df2.Away == away]['cv5_sofridos_away_ft'].iloc[0]
            jogo['avg5_away_05ft'] = df2[df2.Away == away]['avg5_away_05ft'].iloc[0]
            jogo['avg5_away_05ht'] = df2[df2.Away == away]['avg5_away_05ht'].iloc[0]
            jogo['avg5_away_15ft'] = df2[df2.Away == away]['avg5_away_15ft'].iloc[0]
            jogo['avg5_away_25ft'] = df2[df2.Away == away]['avg5_away_25ft'].iloc[0]
            jogo['avg5_away_35ft'] = df2[df2.Away == away]['avg5_away_35ft'].iloc[0]
            jogo['avg5_away_45ft'] = df2[df2.Away == away]['avg5_away_45ft'].iloc[0]
            jogo['avg5_away_btts_ft'] = df2[df2.Away == away]['avg5_away_btts_ft'].iloc[0]
            jogo['mean10_gols_feitos_away_ft'] = df2[df2.Away == away]['mean10_gols_feitos_away_ft'].iloc[0]
            jogo['cv10_feitos_away_ft'] = df2[df2.Away == away]['cv10_feitos_away_ft'].iloc[0]
            jogo['mean10_gols_sofridos_away_ft'] = df2[df2.Away == away]['mean10_gols_sofridos_away_ft'].iloc[0]
            jogo['cv10_sofridos_away_ft'] = df2[df2.Away == away]['cv10_sofridos_away_ft'].iloc[0]
            jogo['avg10_away_05ft'] = df2[df2.Away == away]['avg10_away_05ft'].iloc[0]
            jogo['avg10_away_05ht'] = df2[df2.Away == away]['avg10_away_05ht'].iloc[0]
            jogo['avg10_away_15ft'] = df2[df2.Away == away]['avg10_away_15ft'].iloc[0]
            jogo['avg10_away_25ft'] = df2[df2.Away == away]['avg10_away_25ft'].iloc[0]
            jogo['avg10_away_35ft'] = df2[df2.Away == away]['avg10_away_35ft'].iloc[0]
            jogo['avg10_away_45ft'] = df2[df2.Away == away]['avg10_away_45ft'].iloc[0]
            jogo['avg10_away_btts_ft'] = df2[df2.Away == away]['avg10_away_btts_ft'].iloc[0]
            jogo['cv_valordogol_away_intervalos'] = df2[df2.Away == away]['cv_valordogol_away_intervalos'].iloc[0]
            jogo['valordogol_away_intervalos' ] = df2[df2.Away == away]['valordogol_away_intervalos' ].iloc[0]  
        
            jogo['coeficiente10_gols_sofridos' ] = df2[df2.Away == away]['coeficiente10_gols_sofridos' ].iloc[0]
            
            jogo['Media_Ptos_A'] = df2[df2.Away == away]['Media_Ptos_A'].iloc[0]
            jogo['CV_Ptos_A'] = df2[df2.Away == away]['CV_Ptos_A'].iloc[0]
            jogo['Media_VG_A'] = df2[df2.Away == away]['Media_VG_A'].iloc[0]
            jogo['CV_VG_A'] = df2[df2.Away == away]['CV_VG_A'].iloc[0]
            jogo['Media_CG_A'] = df2[df2.Away == away]['Media_CG_A'].iloc[0]
            jogo['CV_CG_A'] = df2[df2.Away == away]['CV_CG_A'].iloc[0]
            
            
            
            
            
        except:
            pass
        
        lista.append(jogo)


df = pd.DataFrame(lista)
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)

# CV das Odds do Match Odds
df['CV_HDA'] = df[['FT_Odd_H','FT_Odd_D','FT_Odd_A']].std(ddof=0, axis=1) / df[['FT_Odd_H','FT_Odd_D','FT_Odd_A']].mean(axis=1)




In [85]:
df['p_H'] = 1 / df.FT_Odd_H
df['p_D'] = 1 / df.FT_Odd_D
df['p_A'] = 1 / df.FT_Odd_A
df['p_Over'] = 1 / df.FT_Odd_Over25

In [86]:
filtro = (   
  ((df.p_H>0.5649717514124294) & (df.p_H<0.9259259259259258))
& ((df.p_D>0.08333333333333333) & (df.p_D<0.3076923076923077))
& ((df.p_A>0.03333333333333333) & (df.p_A<0.2777777777777778))
& ((df.p_Over>0.3472222222222222) & (df.p_Over<0.7751937984496123))
& ((df.CV_HDA>0.36234682380465133) & (df.CV_HDA<1.159609030595692))
& ((df.Media_Ptos_H>0.2) & (df.Media_Ptos_H<3.0))
& ((df.Media_Ptos_A>0.0) & (df.Media_Ptos_A<1.75))
& ((df.CV_Ptos_H>0.0) & (df.CV_Ptos_H<2.2360679774997907))
& ((df.CV_Ptos_A>0.0) & (df.CV_Ptos_A<2.236067977499792))
& ((df.Media_VG_H>0.059340659340659074) & (df.Media_VG_H<1.0757265992756089))
& ((df.Media_VG_A>0.1934731934731934) & (df.Media_VG_A<1.7645676691729317))
& ((df.CV_VG_H>0.06332299533013878) & (df.CV_VG_H<1.8339629103004964))
& ((df.CV_VG_A>0.06869644114659167) & (df.CV_VG_A<1.5770968944078807))
& ((df.Media_CG_H>0.10727190605239376) & (df.Media_CG_H<0.675248419150858))
& ((df.Media_CG_A>0.10557290573168168) & (df.Media_CG_A<0.298245614035088))
& ((df.CV_CG_H>0.0) & (df.CV_CG_H<1.096295307426663))
& ((df.CV_CG_A>0.0) & (df.CV_CG_A<1.2784658672552298))
)

#Dataframe Filtrado
def retorna_df(dicionario,data):
    data = data.copy()
    list_columns = dicionario.keys()
    for i in data.columns.to_list():
        for x in list_columns:
            if i.lower() in x:
                try:
                    x_min = dicionario[f'{i.lower()}_min']
                    x_max = dicionario[f'{i.lower()}_max']
                    data = data[data[i].between(x_min,x_max)]
                except:
                    pass
    return data

result = {'p_h_min': 0.14285714285714285,
 'p_h_max': 0.31746031746031744,
 'p_d_min': 0.2,
 'p_d_max': 0.3333333333333333,
 'p_a_min': 0.5,
 'p_a_max': 0.6535947712418301,
 'p_over_min': 0.38167938931297707,
 'p_over_max': 0.6711409395973155,
 'cv_hda_min': 0.23750435276082887,
 'cv_hda_max': 0.6142425378260569,
 'media_ptos_h_min': 0.0,
 'media_ptos_h_max': 2.6,
 'media_ptos_a_min': 0.2,
 'media_ptos_a_max': 3.0,
 'cv_ptos_h_min': 0.0,
 'cv_ptos_h_max': 2.23606797749979,
 'cv_ptos_a_min': 0.0,
 'cv_ptos_a_max': 2.23606797749979,
 'media_vg_h_min': 0.09753694581280767,
 'media_vg_h_max': 0.9479747305834264,
 'media_vg_a_min': 0.1706766917293232,
 'media_vg_a_max': 0.7879256965944271,
 'cv_vg_h_min': 0.12487264529334759,
 'cv_vg_h_max': 1.8308606668040055,
 'cv_vg_a_min': 0.13251514240141918,
 'cv_vg_a_max': 1.7934349826132214,
 'media_cg_h_min': 0.10220797720797693,
 'media_cg_h_max': 0.5793650793650793,
 'media_cg_a_min': 0.11045270125987595,
 'media_cg_a_max': 0.5484167837555098,
 'cv_cg_h_min': 0.015890040026663988,
 'cv_cg_h_max': 1.0965284591499063,
 'cv_cg_a_min': 0.02119806196276601,
 'cv_cg_a_max': 0.9397910869035022}


Lay_1x0 = retorna_df(result,df)
Lay_1x0  = Lay_1x0 [['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                       'FT_Odd_Over25', 'FT_Odd_BTTS_Yes', 'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
Lay_1x0.reset_index(inplace=True, drop=True)
Lay_1x0.index = Lay_1x0.index.set_names(['Nº'])
Lay_1x0 = Lay_1x0.rename(index=lambda x: x + 1)
Lay_1x0 = Lay_1x0.sort_values(['Date'])
Lay_1x0.reset_index(inplace=True, drop=True)
Lay_1x0.index = Lay_1x0.index.set_names(['Nº'])
Lay_1x0 = Lay_1x0.rename(index=lambda x: x + 1)

In [87]:
under4_5 = (((df['avg10_home_45ft'] == 0) & (df['avg10_away_45ft'] == 0))
& ((df['avg10_home_25ft'] <= 10) | (df['avg10_away_25ft'] <= 10)))

under4_5 = df[under4_5]

under4_5  = under4_5 [['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                       'FT_Odd_Over25', 'FT_Odd_BTTS_Yes', 'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]

under4_5.reset_index(inplace=True, drop=True)
under4_5.index =  under4_5.index.set_names(['Nº'])
under4_5 =  under4_5.rename(index=lambda x: x + 1)
under4_5 =  under4_5.sort_values(['Date'])
under4_5.reset_index(inplace=True, drop=True)
under4_5.index =  under4_5.index.set_names(['Nº'])
under4_5 =  under4_5.rename(index=lambda x: x + 1)


In [88]:
filtro = (   
  ((df.p_H>0.5649717514124294) & (df.p_H<0.9259259259259258))
& ((df.p_D>0.08333333333333333) & (df.p_D<0.3076923076923077))
& ((df.p_A>0.03333333333333333) & (df.p_A<0.2777777777777778))
& ((df.p_Over>0.3472222222222222) & (df.p_Over<0.7751937984496123))
& ((df.CV_HDA>0.36234682380465133) & (df.CV_HDA<1.159609030595692))
& ((df.Media_Ptos_H>0.2) & (df.Media_Ptos_H<3.0))
& ((df.Media_Ptos_A>0.0) & (df.Media_Ptos_A<1.75))
& ((df.CV_Ptos_H>0.0) & (df.CV_Ptos_H<2.2360679774997907))
& ((df.CV_Ptos_A>0.0) & (df.CV_Ptos_A<2.236067977499792))
& ((df.Media_VG_H>0.059340659340659074) & (df.Media_VG_H<1.0757265992756089))
& ((df.Media_VG_A>0.1934731934731934) & (df.Media_VG_A<1.7645676691729317))
& ((df.CV_VG_H>0.06332299533013878) & (df.CV_VG_H<1.8339629103004964))
& ((df.CV_VG_A>0.06869644114659167) & (df.CV_VG_A<1.5770968944078807))
& ((df.Media_CG_H>0.10727190605239376) & (df.Media_CG_H<0.675248419150858))
& ((df.Media_CG_A>0.10557290573168168) & (df.Media_CG_A<0.298245614035088))
& ((df.CV_CG_H>0.0) & (df.CV_CG_H<1.096295307426663))
& ((df.CV_CG_A>0.0) & (df.CV_CG_A<1.2784658672552298))
)

Lay_0x1 = df[filtro]

Lay_0x1  = Lay_0x1 [['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                       'FT_Odd_Over25', 'FT_Odd_BTTS_Yes', 'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]

Lay_0x1.reset_index(inplace=True, drop=True)
Lay_0x1.index =  Lay_0x1.index.set_names(['Nº'])
Lay_0x1 =  Lay_0x1.rename(index=lambda x: x + 1)
Lay_0x1 =  Lay_0x1.sort_values(['Date'])
Lay_0x1.reset_index(inplace=True, drop=True)
Lay_0x1.index =  Lay_0x1.index.set_names(['Nº'])
Lay_0x1 =  Lay_0x1.rename(index=lambda x: x + 1)


In [89]:
flt = (
((df.CV_HDA >= 0.3) & (df.CV_HDA <= 0.4))
& ((df.FT_Odd_H >= 1.01) & (df.FT_Odd_H <= 2.01))
& ((df.FT_Odd_D >= 3))
& ((df.FT_Odd_Over25 >= 1.01) & (df.FT_Odd_Over25 <= 2.01))
& ((df.FT_Odd_BTTS_Yes >= 1.45))
& ((df.Media_CG_H <= 0.31))
)

Back_Draw = df[flt]
Back_Draw = Back_Draw[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                       'FT_Odd_Over25', 'FT_Odd_BTTS_Yes', 'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
Back_Draw = drop_reset_index(Back_Draw)



metodo_1_5ft =  ((df['avg10_home_15ft'] >= 50) & (df['avg10_away_15ft'] >= 50)
& (df['mean10_gols_feitos_home_ft'] >= 1.0) & (df['cv10_feitos_home_ft'] <= 1)
& (df['mean10_gols_sofridos_away_ft'] >= 1.0) & (df['cv10_sofridos_away_ft'] <= 1)
& (df['mean10_gols_feitos_away_ft'] >= 1.0) & (df['cv10_feitos_away_ft'] <= 1)
& (df['mean10_gols_sofridos_home_ft'] >= 1.0) & (df['cv10_sofridos_home_ft'] <= 1)                               
& (df['avg10_away_btts_ft'] >= 50) & (df['avg10_home_btts_ft'] >= 50))

over1_5_ft_futpython = df[metodo_1_5ft]

over1_5_ft_futpython= over1_5_ft_futpython[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                       'FT_Odd_Over25', 'FT_Odd_BTTS_Yes', 'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
over1_5_ft_futpython = drop_reset_index(over1_5_ft_futpython)

    
metodo_2_5ft = ((df['avg10_home_25ft'] >= 50) & (df['avg10_away_25ft'] >= 50)
& (df['mean10_gols_feitos_home_ft'] >= 1.0) & (df['cv10_feitos_home_ft'] <= 1)
& (df['mean10_gols_sofridos_away_ft'] >= 1.0) & (df['cv10_sofridos_away_ft'] <= 1)
& (df['mean10_gols_feitos_away_ft'] >= 1.0) & (df['cv10_feitos_away_ft'] <= 1)
& (df['mean10_gols_sofridos_home_ft'] >= 1.0) & (df['cv10_sofridos_home_ft'] <= 1)                               
& (df['avg10_away_btts_ft'] >= 50) & (df['avg10_home_btts_ft'] >= 50))

over2_5_ft_futpython = df[metodo_2_5ft]

over2_5_ft_futpython= over2_5_ft_futpython[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                       'FT_Odd_Over25', 'FT_Odd_BTTS_Yes', 'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
over2_5_ft_futpython = drop_reset_index(over2_5_ft_futpython)


flt2= (
((df.valordogol_home_intervalos >=4) & (df.valordogol_away_intervalos >= 4))
)

Over25_15 = df[flt2]
Over25_15 = Over25_15[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                       'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
Over25_15 = drop_reset_index(Over25_15)

flt3= (
((df.cv_valordogol_home_intervalos <=2) & (df.cv_valordogol_away_intervalos <= 2))
)

btts_yes = df[flt3]
btts_yes = btts_yes[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                     'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
btts_yes = drop_reset_index(btts_yes)


flt4= (
((df.cv_valordogol_home_intervalos >=5) & (df.cv_valordogol_away_intervalos >= 5))
)

btts_no = df[flt4]
btts_no = btts_no[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                   'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
btts_no = drop_reset_index(btts_no)


lay_goleada = (df['coeficiente10_gols_feitos'].between(-0.6, 0.6) & (df['cv10_gols_feitos'] <= 2))
lay_goleada = df[lay_goleada]
lay_goleada = lay_goleada[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                           'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
lay_goleada = drop_reset_index(lay_goleada)


selecao05 = ((df['avg5_home_05ht'] >= 80) & (df['avg5_away_05ht'] >= 80) &
             (df['avg10_home_05ht'] >= 60) & (df['avg10_away_05ht'] >= 60) &
             (df['avg5_home_15ft'] >= 80) & (df['avg5_away_15ft'] >= 80) &
             (df['avg10_home_15ft'] >= 80) & (df['avg10_away_15ft'] >= 80) &
             ((df['mean55_gols_feitos_home_ft'] + df['mean55_gols_sofridos_home_ft']) >= 1) &
             ((df['mean55_gols_feitos_away_ft'] + df['mean55_gols_sofridos_away_ft']) >= 1))

over05ht = df[selecao05]
over05ht = over05ht[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                     'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
over05ht = drop_reset_index(over05ht)


lay_away = ((df['coeficiente_pontos'] >= 0.6) & (df['cv_pontos_intervalos'] <= 2) &
             (df['FT_Odd_H'] < df['FT_Odd_A']) & (df['FT_Odd_A'] <= 6))

lay_away = df[lay_away]
lay_away = lay_away[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                     'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
lay_away = drop_reset_index(lay_away)


lay_home =(df['FT_Odd_H'].between(2.4, 4.5) & (df['CV_VG_H'] >= 0.96))
lay_home = df[lay_home]

lay_home = lay_home[['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                     'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
lay_home = drop_reset_index(lay_home)



over2_5 = ((df['avg10_home_25ft'] >= 60) & (df['avg10_away_25ft'] >= 60))
over2_5 = df[over2_5]
over2_5  = over2_5 [['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                     'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
over2_5  = drop_reset_index(over2_5 )


lay_away_new = ((df['coeficiente_pontos'] >= 0.6) & (df['cv_pontos_intervalos'] <= 4) &
                 (df['coeficiente10_gols_feitos'] > 0) & (df['coeficiente10_gols_sofridos'] < 0))

lay_away_new = df[lay_away_new]

lay_away_new  = lay_away_new [['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                               'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
lay_away_new  = drop_reset_index(lay_away_new)

lay_home_new = ((df['cv_pontos'] <= 2.49) & (df['CV_VG_H'] >= 0.79)
             & (df['cv10_gols_feitos'] <= 1.85))

lay_home_new = df[lay_home_new]
lay_home_new  = lay_home_new [['Date','Time','League','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A', 
                               'FT_Odd_Over25', 'FT_Odd_BTTS_Yes']]
lay_home_new  = drop_reset_index(lay_home_new)

under4_5 = under4_5.sort_values(['Time'])
Lay_0x1 = Lay_0x1.sort_values(['Time'])
Lay_1x0 = Lay_1x0.sort_values(['Time'])
over2_5_ft_futpython = over2_5_ft_futpython.sort_values(['Time'])
over1_5_ft_futpython = over1_5_ft_futpython.sort_values(['Time'])
lay_home_new = lay_home_new.sort_values(['Time'])
lay_away_new = lay_away_new.sort_values(['Time'])
over2_5 = over2_5.sort_values(['Time'])
lay_away_new = lay_away_new.sort_values(['Time'])
lay_home = lay_home.sort_values(['Time'])
over05ht = over05ht.sort_values(['Time'])
lay_goleada = lay_goleada.sort_values(['Time'])
btts_no = btts_no.sort_values(['Time'])
btts_yes = btts_yes.sort_values(['Time'])
Over25_15 = Over25_15.sort_values(['Time'])
Back_Draw = Back_Draw.sort_values(['Time'])


#caminho = 'C:/Users/thiag/OneDrive/curso python/modelo_leandro_2/flashscore/'
#df.to_excel(caminho + 'jogos_dia_completo_arkad.xlsx', sheet_name='completo', encoding='utf-8', index=False)
# + timedelta(0)
#data = datetime.today() + timedelta(0)
hoje = data.strftime('%Y-%m-%d')
nome = 'Jogos_do_Dia_arkad_transform(last).xlsx'
Jogos_do_Dia.to_excel(f'{hoje}_{nome}')
writer=pd.ExcelWriter(f'{hoje}_{nome}',engine='xlsxwriter' )
#writer=pd.ExcelWriter(caminho + 'jogos_dia_completo_arkad.xlsx', engine='xlsxwriter')

df.to_excel(writer, sheet_name='Completo', encoding='utf-8', index=False)

lay_away_new.to_excel(writer, sheet_name='lay_away_new',
                        encoding='utf-8', index=False)

lay_home_new.to_excel(writer, sheet_name='lay_home_new',
                        encoding='utf-8', index=False)

btts_yes.to_excel(writer, sheet_name='btts_yes',
                        encoding='utf-8', index=False)

over2_5.to_excel(writer, sheet_name='over2_5',
                        encoding='utf-8', index=False)

lay_goleada.to_excel(writer, sheet_name='lay_goleada',
                        encoding='utf-8', index=False)

lay_away.to_excel(writer, sheet_name='lay_away',
                        encoding='utf-8', index=False)

lay_home.to_excel(writer, sheet_name='lay_home',
                        encoding='utf-8', index=False)

Over25_15.to_excel(writer, sheet_name='Over25_15',
                        encoding='utf-8', index=False)
over05ht.to_excel(writer, sheet_name='over05ht',
                        encoding='utf-8', index=False)

over2_5_ft_futpython.to_excel(writer, sheet_name='over2_5_ft_futpython',
                        encoding='utf-8', index=False)

over1_5_ft_futpython.to_excel(writer, sheet_name='over1_5_ft_futpython',
                        encoding='utf-8', index=False)

Lay_0x1.to_excel(writer, sheet_name='Lay_0x1',
                        encoding='utf-8', index=False)

Lay_1x0.to_excel(writer, sheet_name='Lay_1x0',
                        encoding='utf-8', index=False)

Back_Draw.to_excel(writer, sheet_name='Back_Draw',
                        encoding='utf-8', index=False)

btts_no.to_excel(writer, sheet_name='btts_no',
                        encoding='utf-8', index=False)

under4_5.to_excel(writer, sheet_name='under4_5',
                        encoding='utf-8', index=False) 

writer.save()
writer.close()




print('################')
print('Encerrado')


################
Encerrado


In [90]:
#+ timedelta(0)
#data = date.today()  + timedelta(0)
hoje = data.strftime('%Y-%m-%d')
nome = 'under4_5.csv'
under4_5 = under4_5.sort_values(['Time'])
under4_5.to_csv(f'{hoje}_{nome}')

nome = 'Lay_0x1.csv'
Lay_0x1 = Lay_0x1.sort_values(['Time'])
Lay_0x1.to_csv(f'{hoje}_{nome}')

nome = 'Lay_1x0.csv'
Lay_1x0 = Lay_1x0.sort_values(['Time'])
Lay_1x0.to_csv(f'{hoje}_{nome}')

over2_5_ft_futpython = over2_5_ft_futpython.sort_values(['Time'])
nome = 'over2_5_ft_futpython.csv'
over2_5_ft_futpython.to_csv(f'{hoje}_{nome}')

over1_5_ft_futpython = over1_5_ft_futpython.sort_values(['Time'])
nome = 'over1_5_ft_futpython.csv'
over1_5_ft_futpython.to_csv(f'{hoje}_{nome}')

lay_home_new = lay_home_new.sort_values(['Time'])
nome = 'lay_home_new.csv'
lay_home_new.to_csv(f'{hoje}_{nome}')

lay_away_new = lay_away_new.sort_values(['Time'])
nome = 'lay_away_new.csv'
lay_away_new.to_csv(f'{hoje}_{nome}')

over2_5 = over2_5.sort_values(['Time'])
nome = 'over2_5.csv'
over2_5.to_csv(f'{hoje}_{nome}')

lay_away_new = lay_away_new.sort_values(['Time'])
nome = 'lay_away.csv'
lay_away_new.to_csv(f'{hoje}_{nome}')

lay_home = lay_home.sort_values(['Time'])
nome = 'lay_home.csv'
lay_home.to_csv(f'{hoje}_{nome}')

over05ht = over05ht.sort_values(['Time'])
nome = 'over05ht.csv'
over05ht.to_csv(f'{hoje}_{nome}')


lay_goleada = lay_goleada.sort_values(['Time'])
nome = 'lay_goleada.csv'
lay_goleada.to_csv(f'{hoje}_{nome}')

btts_no = btts_no.sort_values(['Time'])
nome = 'btts_no.csv'
btts_no.to_csv(f'{hoje}_{nome}')

btts_yes = btts_yes.sort_values(['Time'])
nome = 'btts_yes.csv'
btts_yes.to_csv(f'{hoje}_{nome}')

Over25_15 = Over25_15.sort_values(['Time'])
nome = 'Over25_15.csv'
Over25_15.to_csv(f'{hoje}_{nome}')

Back_Draw = Back_Draw.sort_values(['Time'])
nome = 'Back_Draw.csv'
Back_Draw.to_csv(f'{hoje}_{nome}')

print("encerrado")

encerrado
